# JS: Viewport Height

> JavaScript generator for dynamic viewport height calculation.

In [ ]:
#| default_exp js.viewport

In [ ]:
#| export
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds

## generate_viewport_height_js

Sets explicit pixel height on the card stack to fill available vertical space.

**Algorithm:**
1. `siblingSpace` — Measure actual sibling elements within the container (header, controls, footer)
2. `containerPadding` — Account for container's top and bottom padding
3. `spaceBelow` — Walk up DOM tree measuring padding/borders and siblings positioned below (not beside)
4. `viewportHeight = windowHeight - containerTop - siblingSpace - containerPadding - spaceBelow`

**Key design decisions:**
- **Direct sibling measurement** (not subtraction) — Works in flex layouts where sibling columns inflate container height
- **Position-based sibling filtering** — Only counts siblings with `top >= currentElement.bottom`, handles flex row layouts
- **Debug mode** — Set `window.cardStackDebug = true` to log all intermediate values

The `container_id` parameter is the consumer's parent container that holds
the card stack alongside header/footer elements. If not provided, the card
stack measures from its own position.

In [ ]:
#| export
def generate_viewport_height_js(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    container_id: str = "",  # Consumer's parent container ID (empty = use card stack parent)
) -> str:  # JavaScript code fragment for viewport height calculation
    """Generate JS for dynamic viewport height calculation.

    Computes sibling space by measuring actual sibling elements within
    the container. This works correctly in flex layouts where sibling
    columns can inflate the container height.

    Debug mode: Set `window.cardStackDebug = true` in browser console
    to log all intermediate calculation values.
    """
    handler_key = f"_csResizeHandler_{ids.prefix.replace('-', '_')}"
    prefix = ids.prefix

    # Build container resolution logic — guarded against null for SPA navigation
    if container_id:
        container_resolution = f"""
            const container = document.getElementById('{container_id}');"""
    else:
        container_resolution = f"""
            const _csEl = document.getElementById('{ids.card_stack}');
            if (!_csEl) return 400;
            const container = _csEl.parentElement;"""

    return f"""
        // === Viewport Height Calculation ===
        const _debug = () => window.cardStackDebug === true;
        const _log = (...args) => {{ if (_debug()) console.log('[{prefix}]', ...args); }};

        function calculateSpaceBelowContainer(container) {{
            // Walk up the DOM tree and measure space used below the container.
            // Only counts siblings that are actually positioned below (not beside
            // in flex rows) by comparing bounding rect positions.
            let spaceUsedBelow = 0;
            let currentElement = container;
            let currentRect = currentElement.getBoundingClientRect();
            let parent = container.parentElement;
            let level = 0;

            while (parent && parent !== document.documentElement) {{
                const parentStyles = getComputedStyle(parent);
                const paddingBottom = parseFloat(parentStyles.paddingBottom) || 0;
                const borderBottom = parseFloat(parentStyles.borderBottomWidth) || 0;
                spaceUsedBelow += paddingBottom + borderBottom;

                _log(`spaceBelow L${{level}}: parent=${{parent.tagName}}#${{parent.id || '(no id)'}}, padding=${{paddingBottom}}, border=${{borderBottom}}`);

                let foundCurrent = false;
                for (const sibling of parent.children) {{
                    if (sibling === currentElement) {{ foundCurrent = true; continue; }}
                    if (!foundCurrent) continue;
                    if (sibling.nodeType !== Node.ELEMENT_NODE) continue;
                    const tag = sibling.tagName;
                    if (tag === 'SCRIPT' || tag === 'STYLE') continue;
                    if (tag === 'INPUT' && sibling.type === 'hidden') continue;
                    const s = getComputedStyle(sibling);
                    if (s.display === 'none') continue;

                    // Only count siblings that are actually positioned below,
                    // not beside (handles flex row layouts)
                    const siblingRect = sibling.getBoundingClientRect();
                    if (siblingRect.top < currentRect.bottom) {{
                        _log(`  skip (beside): ${{tag}}#${{sibling.id || '(no id)'}}, top=${{siblingRect.top}} < bottom=${{currentRect.bottom}}`);
                        continue;
                    }}

                    const mt = parseFloat(s.marginTop) || 0;
                    const mb = parseFloat(s.marginBottom) || 0;
                    spaceUsedBelow += siblingRect.height + mt + mb;
                    _log(`  below: ${{tag}}#${{sibling.id || '(no id)'}}, h=${{siblingRect.height}}, margins=${{mt}}/${{mb}}`);
                }}

                currentElement = parent;
                currentRect = currentElement.getBoundingClientRect();
                parent = parent.parentElement;
                level++;
            }}
            return spaceUsedBelow;
        }}

        function calculateSiblingSpace(container, cardStackId) {{
            // Measure actual sibling elements within the container.
            // This works correctly in flex layouts where sibling columns
            // can inflate the container height.
            let siblingSpace = 0;
            for (const child of container.children) {{
                if (child.id === cardStackId) continue;
                if (child.nodeType !== Node.ELEMENT_NODE) continue;
                const tag = child.tagName;
                if (tag === 'SCRIPT' || tag === 'STYLE') continue;
                if (tag === 'INPUT' && child.type === 'hidden') continue;
                const s = getComputedStyle(child);
                if (s.display === 'none') continue;
                const rect = child.getBoundingClientRect();
                const mt = parseFloat(s.marginTop) || 0;
                const mb = parseFloat(s.marginBottom) || 0;
                siblingSpace += rect.height + mt + mb;
                _log(`sibling: ${{tag}}#${{child.id || '(no id)'}}, h=${{rect.height}}, margins=${{mt}}/${{mb}}`);
            }}
            return siblingSpace;
        }}

        function calculateAndSetViewportHeight() {{
            {container_resolution}
            const cardStack = document.getElementById('{ids.card_stack}');
            const cardStackInner = document.getElementById('{ids.card_stack_inner}');
            if (!container || !cardStack) {{
                _log('ERROR: container or cardStack not found');
                return 400;
            }}

            _log('=== calculateAndSetViewportHeight ===');
            _log('container:', container.id || '(no id)');

            // Measure actual siblings within the container
            const siblingSpace = calculateSiblingSpace(container, '{ids.card_stack}');

            // Account for container padding
            const containerStyle = getComputedStyle(container);
            const containerPaddingTop = parseFloat(containerStyle.paddingTop) || 0;
            const containerPaddingBottom = parseFloat(containerStyle.paddingBottom) || 0;
            const containerPadding = containerPaddingTop + containerPaddingBottom;

            const windowHeight = window.innerHeight;
            const containerTop = container.getBoundingClientRect().top;
            const spaceBelow = calculateSpaceBelowContainer(container);

            const rawHeight = windowHeight - containerTop - siblingSpace - containerPadding - spaceBelow;
            const viewportHeight = Math.floor(Math.max(200, rawHeight));

            _log('windowHeight:', windowHeight);
            _log('containerTop:', containerTop);
            _log('siblingSpace:', siblingSpace);
            _log('containerPadding:', containerPadding, `(${{containerPaddingTop}} + ${{containerPaddingBottom}})`);
            _log('spaceBelow:', spaceBelow);
            _log('rawHeight:', rawHeight);
            _log('viewportHeight:', viewportHeight, rawHeight < 200 ? '(clamped to min)' : '');

            cardStack.style.height = viewportHeight + 'px';
            cardStack.style.maxHeight = viewportHeight + 'px';
            cardStack.style.minHeight = viewportHeight + 'px';
            if (cardStackInner) cardStackInner.style.height = viewportHeight + 'px';

            return viewportHeight;
        }}

        function _debounce(fn, delay) {{
            let tid;
            return function(...args) {{ clearTimeout(tid); tid = setTimeout(() => fn.apply(this, args), delay); }};
        }}

        const _debouncedResize = _debounce(function() {{
            calculateAndSetViewportHeight();
            if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
        }}, 100);

        // Remove old resize listener from previous IIFE (handles HTMX page
        // navigation that re-executes this script without a full page reload).
        if (window.{handler_key}) {{
            window.removeEventListener('resize', window.{handler_key});
        }}
        window.{handler_key} = _debouncedResize;
        window.addEventListener('resize', _debouncedResize);

        // Expose on namespace
        ns.recalculateHeight = calculateAndSetViewportHeight;
    """

In [ ]:
# Test that the generator produces valid JS referencing correct IDs
ids = CardStackHtmlIds(prefix="cs0")
js = generate_viewport_height_js(ids, container_id="my-container")
assert "my-container" in js
assert ids.card_stack in js
assert ids.card_stack_inner in js
assert "ns.recalculateHeight" in js
assert "calculateAndSetViewportHeight" in js
# Verify direct sibling measurement approach
assert "calculateSiblingSpace" in js
assert "siblingSpace" in js
assert "spaceBelow" in js
# Verify container padding is accounted for
assert "containerPadding" in js
assert "paddingTop" in js
assert "paddingBottom" in js
# Verify flex row handling (position check for siblings)
assert "siblingRect.top < currentRect.bottom" in js
# Verify debug mode
assert "window.cardStackDebug" in js
assert f"[{ids.prefix}]" in js  # debug log prefix
# Verify auto-adjust trigger in resize handler
assert "ns.triggerAutoAdjust" in js
# Verify resize listener uses remove-and-replace (not guard-and-skip)
handler_key = f"_csResizeHandler_{ids.prefix.replace('-', '_')}"
assert f"window.{handler_key}" in js
assert "removeEventListener" in js
print("Viewport height JS generator tests passed!")

Viewport height JS generator tests passed!


In [ ]:
# Test without container_id (falls back to parent element with null guard)
js_no_container = generate_viewport_height_js(ids)
assert ".parentElement" in js_no_container
# Verify null guard exists before .parentElement access
assert "if (!_csEl) return 400" in js_no_container
# Verify direct sibling measurement is present in both paths
assert "siblingSpace" in js_no_container
assert "calculateSiblingSpace" in js_no_container
print("Viewport height JS fallback container test passed!")

Viewport height JS fallback container test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()